# Initialize

### Hugging Face

In [ ]:
#get token from https://huggingface.co/
from huggingface_hub import interpreter_login

interpreter_login()

### Connect to Drive

Required to save results in Cloud

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### get dependencies and load repos

Loads threestudio, Wonder3D, and Evaluate3D (including CLIP and trimesh) from https://github.com/threestudio-project/threestudio.git

#### FOR THREESTUDIO:
go to \\
/content/threestudio/threestudio/models/exporters/mesh_exporter.py (line 30) \\
/content/threestudio/threestudio/models/renderers/nvdiff_rasterizer.py (line 21) \\
and change context_type: str = "gl" to context_type: str = "cuda"  \\

Also, go to \\
/content/threestudio/threestudio/utils/rasterize.py \\
and add context_type: str = "cuda" between line 9 and 10 \\

save all changes (Strg + S); can already be done after Threestudio Github was cloned and other dependencies are still loading\\

Additionally, change config files in to increase iteration steps etc \\

DreamFusion: /content/threestudio/configs/dreamfusion-if.yaml \\
Magic3D: /content/threestudio/configs/magic3d-coarse-if.yaml and /content/threestudio/configs/magic3d-refine-sd.yaml \\
Fantasia3D: /content/threestudio/configs/fantasia3d.yaml and /content/threestudio/configs/fantasia3d-texture.yaml \\
Magic123: /content/threestudio/configs/magic123-coarse-sd.yaml and /content/threestudio/configs/magic123-refine-sd.yaml

In [ ]:
import os
!git clone https://github.com/threestudio-project/threestudio.git
%cd threestudio

!pip install ninja
!pip install lightning==2.0.0 omegaconf==2.3.0 jaxtyping typeguard diffusers transformers accelerate opencv-python tensorboard matplotlib imageio imageio[ffmpeg] trimesh bitsandbytes sentencepiece safetensors huggingface_hub libigl xatlas networkx pysdf PyMCubes wandb torchmetrics controlnet_aux
!pip install einops kornia taming-transformers-rom1504 git+https://github.com/openai/CLIP.git
!pip install open3d
!pip install mediapipe
!pip install "diffusers<0.20"
!pip install trimesh
!pip install pygltflib

!pip install git+https://github.com/ashawkey/envlight.git
!pip install git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2  # ~5min
!pip install git+https://github.com/NVlabs/nvdiffrast.git
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch  # ~20-30min

### Prompt

In [ ]:
prompt = "a symmetrical tennis ball"

#All Models

## Threestudio

###Dreamfusion-if

In [ ]:
# Train
!python launch.py --config configs/dreamfusion-if.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt"

In [ ]:
# Export Mesh (maybe change system.geometry_convert_override.isosurface_threshold=X; increase X if result is extruded, decrease if incomplete; auto = 25) <- does not always work as expected
!python launch.py --config /content/threestudio/outputs/dreamfusion-if/a_symmetrical_tennis_ball@20231214-212703/configs/parsed.yaml --export --gpu 0 resume=/content/threestudio/outputs/dreamfusion-if/a_symmetrical_tennis_ball@20231214-212703/ckpts/last.ckpt system.exporter_type=mesh-exporter system.geometry.isosurface_threshold=auto

In [ ]:
!cp -r /content/threestudio/outputs/dreamfusion-if/ /content/drive/MyDrive/threestudio/

###Magic3D

In [ ]:
# Train Base
!python launch.py --config configs/magic3d-coarse-if.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt"

In [ ]:
# Copy coarse result to Drive
!cp -r /content/threestudio/outputs/magic3d-coarse-if/ /content/drive/MyDrive/threestudio/

In [ ]:
# Train Refine
!python launch.py --config configs/magic3d-refine-sd.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt" system.geometry_convert_from=/content/threestudio/outputs/magic3d-coarse-if/a_symmetrical_tennis_ball@20231214-221323/ckpts/last.ckpt

In [ ]:
# Export Mesh Refine
!python launch.py --config /content/threestudio/outputs/magic3d-refine-sd/a_symmetrical_tennis_ball@20231214-224124/configs/parsed.yaml --export --gpu 0 resume=/content/threestudio/outputs/magic3d-refine-sd/a_symmetrical_tennis_ball@20231214-224124/ckpts/last.ckpt system.exporter_type=mesh-exporter system.geometry_convert_override.isosurface_threshold=auto

In [ ]:
# Copy refined result to drive
!cp -r /content/threestudio/outputs/magic3d-refine-sd/ /content/drive/MyDrive/threestudio/

###Fantasia3D

In [ ]:
# Learn the model - geometry stage
# Apply fitting Geometry via: system.geometry.shape_init=ellipsoid system.geometry.shape_init_params="[0.3,0.3,0.3]" [depth, width, height]
!python launch.py --config configs/fantasia3d.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt"

In [ ]:
# Load from existing mesh                                                                                                                                                                                                                                                                                                       #y z, z y, z x
#!python launch.py --config configs/fantasia3d.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt" system.geometry.shape_init=mesh:load/shapes/human.obj system.geometry.shape_init_params=0.9 system.geometry.shape_init_mesh_up=+y system.geometry.shape_init_mesh_front=+z


In [ ]:
!cp -r /content/threestudio/outputs/fantasia3d/ /content/drive/MyDrive/threestudio/

In [ ]:
# Create Texture for Model - appearance stage
!python launch.py --config configs/fantasia3d-texture.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt" system.geometry_convert_from=/content/threestudio/outputs/fantasia3d/a_symmetrical_tennis_ball@20231215-003649/ckpts/last.ckpt

In [ ]:
# Export as mesh
!python launch.py --config /content/threestudio/outputs/fantasia3d-texture/a_symmetrical_tennis_ball@20231215-005804/configs/parsed.yaml --export --gpu 0 resume=/content/threestudio/outputs/fantasia3d-texture/a_symmetrical_tennis_ball@20231215-005804/ckpts/last.ckpt system.exporter_type=mesh-exporter

In [ ]:
# Copy files to Drive
!cp -r /content/threestudio/outputs/fantasia3d-texture/ /content/drive/MyDrive/threestudio/

###Magic123

In [ ]:
# Train coarse
!python launch.py --config configs/magic123-coarse-sd.yaml --train --gpu 0 data.image_path=/content/drive/MyDrive/Images/Ball.png system.prompt_processor.prompt="$prompt"

In [ ]:
!cp -r /content/threestudio/outputs/magic123-coarse-sd/ /content/drive/MyDrive/threestudio/

In [ ]:
# Refine
!python launch.py --config configs/magic123-refine-sd.yaml --train --gpu 0 data.image_path=/content/drive/MyDrive/Images/Ball.png system.prompt_processor.prompt="$prompt" system.geometry_convert_from=/content/threestudio/outputs/magic123-coarse-sd/Ball.png-a_symmetrical_tennis_ball@20231215-015154/ckpts/last.ckpt

In [ ]:
# Export Refine
!python launch.py --config /content/threestudio/outputs/magic123-refine-sd/Ball.png-a_symmetrical_tennis_ball@20231215-023142/configs/parsed.yaml --export --gpu 0 resume=/content/threestudio/outputs/magic123-refine-sd/Ball.png-a_symmetrical_tennis_ball@20231215-023142/ckpts/last.ckpt system.exporter_type=mesh-exporter

In [ ]:
!cp -r /content/threestudio/outputs/magic123-refine-sd/ /content/drive/MyDrive/threestudio/

# Other Models (Threestudio)

##Latentnerf

In [ ]:
# Train in Stable Diffusion latent space
!python launch.py --config configs/latentnerf.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt"

2023-11-27 22:14:11.776676: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 22:14:11.776727: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 22:14:11.776751: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 22:14:12.816427: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is

In [ ]:
# Train in RGB space
!python launch.py --config configs/latentnerf-refine.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt" system.weights=/content/threestudio/outputs/latentnerf/a_detailed_green_fern_in_a_wooden_pot@20231127-221414/ckpts/last.ckpt

2023-11-27 22:52:30.157737: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 22:52:30.157792: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 22:52:30.157819: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 22:52:31.191976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is

In [ ]:
# Export
!python launch.py --config /content/threestudio/outputs/latentnerf-refine/a_detailed_green_fern_in_a_wooden_pot@20231127-225232/configs/parsed.yaml --export --gpu 0 resume=/content/threestudio/outputs/latentnerf-refine/a_detailed_green_fern_in_a_wooden_pot@20231127-225232/ckpts/last.ckpt system.exporter_type=mesh-exporter system.geometry.isosurface_threshold=auto

2023-11-28 00:07:01.520793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 00:07:01.520847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 00:07:01.520873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 00:07:02.562538: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is

In [ ]:
!cp -r /content/threestudio/outputs/latentnerf/ /content/drive/MyDrive/threestudio/
!cp -r /content/threestudio/outputs/latentnerf-refine/ /content/drive/MyDrive/threestudio/

In [ ]:
# train Sketch-Shape in Stable Diffusion latent space using existing .obj
!python launch.py --config configs/sketchshape.yaml --train --gpu 0 system.guide_shape=/load/shapes/teddy.obj system.prompt_processor.prompt="$prompt"

In [ ]:
# refine Sketch-Shape in RGB space
!python launch.py --config configs/sketchshape-refine.yaml --train --gpu 0 system.guide_shape=/content/threestudio/load/shapes/human.obj system.prompt_processor.prompt="$prompt" system.weights=path/to/latent/stage/trial/dir/ckpts/last.ckpt

In [ ]:
!cp -r /content/threestudio/outputs/sketchshape/ /content/drive/MyDrive/threestudio/sketchshape/
!cp -r /content/threestudio/outputs/sketchshape-refine/ /content/drive/MyDrive/threestudio/sketchshape-refine/